In [2]:
# Libreria importantes para la proyecto
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Se corre una vez para mezclar el dataset y se usa el mezclado con la semilla 42
dataSet = pd.read_csv("diabetes.csv")
dataSet = dataSet.sample(frac=1,random_state=42).reset_index(drop=True)

# Se selecionan las columnas con 0 para cambiarlo por la media con los datos que no tienen 0
selected_columns = ['Glucose','BloodPressure', 'SkinThickness', 'Insulin','BMI']  

for col in selected_columns:
    mean_val = dataSet.loc[dataSet[col] != 0, col].mean()
    dataSet[col] = dataSet[col].replace(0, mean_val)

#Se seleciona la columna target y las columnas de features
target = ['Outcome'] 
features = list(set(list(dataSet.columns))-set(target))


#Se guarda el csv para verificar el shuffle y que se hayan cambiado correctametne los valores
dataSet.to_csv("datasetSinCeros.csv", index=False)
dataSet = pd.read_csv("datasetSinCeros.csv")


#Se normalizan los datos en un rango de 0 a 1 por la diferencia de tamaño en las columnas
scaler = MinMaxScaler()
features_to_normalize = dataSet.columns.difference(["Outcome"])
datos_normalizados=pd.DataFrame(scaler.fit_transform(dataSet[features_to_normalize]), columns=features_to_normalize)

datos_normalizados["Outcome"]=dataSet["Outcome"]
datos_normalizados.to_csv("datasetSinCerosN.csv", index=False)

dataSet.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.435949,12.096346,8.790942,85.021108,6.875151,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.750000,64.000000,25.000000,121.500000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.202592,29.153420,155.548223,32.400000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,155.548223,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
datos_normalizados.describe()

,Age,BMI,BloodPressure,DiabetesPedigreeFunction,Glucose,Insulin,Pregnancies,SkinThickness,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,0.204015,0.291564,0.493930,0.168179,0.501205,0.170130,0.226180,0.240798,0.348958
std,0.196004,0.140596,0.123432,0.141473,0.196361,0.102189,0.198210,0.095554,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.050000,0.190184,0.408163,0.070773,0.359677,0.129207,0.058824,0.195652,0.000000
50%,0.133333,0.290389,0.491863,0.125747,0.470968,0.170130,0.176471,0.240798,0.000000
75%,0.333333,0.376278,0.571429,0.234095,0.620968,0.170130,0.352941,0.271739,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
#Se hace el split de los datos 80 entrenamiento ,  20 pruebas, asegurandose el balaceo correcto de datos
#con el parametro adicional stratify
dataTraining, dataTesting = train_test_split(datos_normalizados, test_size=0.2, random_state=42, stratify=dataSet['Outcome'])

#Se asigana el target y los features
x_train = dataTraining[features].values
y_train = dataTraining[target].values


x_test = dataTesting[features].values
y_test = dataTesting[target].values


## el default del alpha es 0.0001
# Se llama a la funcion de MLPCLAssifier que es la red neuronal, y aqui es donde se agustan los hiperparmetros.
mlp = MLPClassifier(hidden_layer_sizes=(64,32,16,8), activation='relu', solver='adam', max_iter=500)


# Se entreba con el vector de features y el vector target que son 80% de los datos
mlp.fit(x_train,y_train)

# Se hace la predicion con el 20% de los datos
predict_train = mlp.predict(x_train)
predict_test = mlp.predict(x_test)
# Se llama a la funcion classification nos da el reporto de metricas
print(classification_report(y_test,predict_test))


c:\Users\tian_\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.83      0.86      0.85       100
           1       0.73      0.69      0.70        54

    accuracy                           0.80       154
   macro avg       0.78      0.77      0.78       154
weighted avg       0.80      0.80      0.80       154

